# Pure Bending of a Curved Bar

In the first file, we had considered the problem of the thick-walled pressure vessel. The solution to the stress field was found by considering a $\theta$-independent Airy stress function. 

In this file, we are going to consider the problem of the pure bending of a curved bar. And, again we are going to consider a $\theta$-independent Airy stress function. 

Before proceeding, however, note an important difference with the first file. In that first file, before going on to the actual problem of the thick-walled pressure vessel, we had written a bunch of definitions to obtain the form of the biharmonic equation in polar coordinates as well as to determine the stress components in polar coordinates through appropriate transformations of the stress fields in the rectangular Cartesian coordinate system. In this file, we do not go into writing all those definitions again. Instead, we copy all those definitions in a separate python file `polarUtilites.py` and store the file in the same folder. So, before proceeding, in order to make those definitions available for the current problem solution, we first need to import that file:

In [15]:
from polarUtilities import *

We now set the Airy stress function to be $\theta$-independent:

In [16]:
phi = sp.Function('phi')(r)
phi

phi(r)

We now solve the biharmonic equation $\nabla^4 \phi = 0$ as follows:

In [17]:
sp.dsolve(polarbiharmonic(phi))

Eq(phi(r), C1 + C2*r**2 + C3*r**2*log(r) + C4*log(r))

This solution is, of course, the same solution that we had obtained in the first file. However, in order to have the same notation as in Timoshenko and Goodier, we rewrite $\phi$ as follows:

In [18]:
A, B, C, D = sp.symbols('A, B, C, D')

phi = A*sp.log(r)+B*r**2*sp.log(r)+C*r**2+D
phi

A*log(r) + B*r**2*log(r) + C*r**2 + D

We again use a better way to display the variables as follows:

In [19]:
from IPython.display import Math, Latex

In [20]:
display(Math(r'\phi = {}'.format(sp.latex(phi))))

<IPython.core.display.Math object>

We can now obtain the expressions of the stress components $\sigma_{rr}$, $\sigma_{\theta\theta}$, and $\sigma_{r\theta}$:

In [21]:
sigmarr = sigma_rr(phi)
sigmatt = sigma_tt(phi)
sigmart = sigma_rt(phi)

display(Math(r'\sigma_{{rr}} = {}'.format(sp.latex(sigmarr))))
display(Math(r'\sigma_{{\theta\theta}} = {}'.format(sp.latex(sigmatt))))
display(Math(r'\sigma_{{r\theta}} = {}'.format(sp.latex(sigmart))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

We note that up to this point there is no difference with the first file. However, we need to solve for $A$, $B$, $C$, and $D$ subject to the boundary conditions appropriate for this problem, which are:

\begin{align}
\sigma_{rr} &= 0 \quad \text{at $r=a$} \\
\sigma_{rr} &= 0 \quad \text{at $r=b$} \\
\int_a^b \sigma_{\theta\theta} \; {\rm d}r &= 0 \\
\int_a^b r \sigma_{\theta\theta} \; {\rm d}r &= -M
\end{align}
where $M$ is the externally applied moment. 

We now set up these boundary conditions as follows:

In [23]:
a, b, M = sp.symbols('a, b, M')

lhs = sigmarr.subs(r,a)
rhs = 0
eq1 = sp.Eq(lhs,rhs)
display(eq1)


lhs = sigmarr.subs(r,b)
rhs = 0
eq2 = sp.Eq(lhs,rhs)
display(eq2)


lhs = sp.integrate(sigmatt,(r,a,b))
rhs = 0
eq3 = sp.Eq(lhs,rhs)
display(eq3)


lhs = sp.integrate(r*sigmatt,(r,a,b))
rhs = -M
eq4 = sp.Eq(lhs,rhs)
display(eq4)

Eq(A/a**2 + 2*B*log(a) + B + 2*C, 0)

Eq(A/b**2 + 2*B*log(b) + B + 2*C, 0)

Eq(A/b - A/a - 2*B*a*log(a) + 2*B*b*log(b) - a*(B + 2*C) + b*(B + 2*C), 0)

Eq(A*log(a) - A*log(b) - B*a**2*log(a) + B*b**2*log(b) - a**2*(B + C) + b**2*(B + C), -M)

Next, we solve for $A$, $B$, and $C$. Note that while we have only three unknowns ($D$ does not appear), we have four equations. Strictly, speaking, we should be using only three of these equations to solve for the three unknowns (one of the equations is not independent). However, operationally speaking, we don't need to worry. A naive implementation using `linsolve` using all four equations still gives us the correct solution; thus:

In [25]:
soln, = sp.linsolve([eq1,eq2,eq3,eq4],(A,B,C))

Avalue = soln[0]
Bvalue = soln[1]
Cvalue = soln[2]

display(Math(r'A = {}'.format(sp.latex(Avalue))))
display(Math(r'B = {}'.format(sp.latex(Bvalue))))
display(Math(r'C = {}'.format(sp.latex(Cvalue))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Now that we have the values of $A$, $B$, and $C$, we can make the appropriate substitutions to find the solutions to the stress components as follows:

In [30]:
sigmarr_soln = sigmarr.subs([(A,Avalue),(B,Bvalue),(C,Cvalue)]).simplify()
sigmatt_soln = sigmatt.subs([(A,Avalue),(B,Bvalue),(C,Cvalue)]).simplify()

display(Math(r'\sigma_{{rr}}^{{\rm soln}} = {}'.format(sp.latex(sigmarr_soln))))
display(Math(r'\sigma_{{\theta\theta}}^{{\rm soln}} = {}'.format(sp.latex(sigmatt_soln))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Note even though these solutions look quite different from those given in Timoshenko and Goodier, they are exactly the same. They may be easily simplified through elementary algebraic identities. 